In [ ]:
__author__ = 'Vinicius Placco <vinicius.placco@noirlab.edu>'
__version__ = '20240118' # yyyymmdd; version datestamp of this notebook
#__datasets__ = ['allwise','smash']
__keywords__ = ['gmos','gemini','stars','dragons']

## Gemini GMOS Photometry reduction using DRAGONS Python API
#### adapted from https://dragons.readthedocs.io/projects/gmosimg-drtutorial/en/v3.1.0/ex1_gmosim_starfield_api.html
#### don't forget to run `conda install nb_conda_kernels ipykernel` to run this notebook on the DRAGONS env

## Table of contents
* [Goals](#goals)
* [Summary](#summary)
* [Disclaimers and attribution](#disclaimer)
* [Imports and setup](#imports)
* [About the dataset](#About)
* [Downloading data for reduction](#Downloading_Data)
* [Set up the DRAGONS logger](#DRAGONS_logger)
* [Create File Lists](#File_Lists)
* [Create Master Bias](#Master_Bias)
* [Create Master Flat Field](#Master_Flat)
* [Reduce Science Images](#Reduce_Science)
* [Display stacked final image](#Display_Image)
* [Clean-up (optional)](#Clean-up)

<a class="anchor" id="goals"></a>
# Goals
Showcase how to perform GMOS imaging data reduction using the Gemini DRAGONS package on the Data Lab science platform. Uses a custom DRAGONS kernel `"DRAGONS (Py3.7)"`. The steps include downloading data from the Gemini archive, setting up a DRAGONS calibration service, processing of bias, flats, fringe, and science frames, and finally the creation of a single combined stacked image.

<a class="anchor" id="summary"></a>
# Summary
DRAGONS is a Python-based astronomical data reduction platform written by the Gemini Science User Support Department. It currently can be used to reduce imaging data from Gemini instruments GMOS, NIRI, Flamingos 2, GSAOI, and GNIRS, and spectroscopic data in GMOS longslit mode. Linked here is a general list of guides, manuals and tutorials about the use of DRAGONS:
https://dragons.readthedocs.io/en/v3.1.0/

The DRAGONS kernel has been made available in the Data Lab environment, which should allow users to access the routines without being dependent on installing the software in their local machines. 

In this notebook, we present an example of a DRAGONS Jupyter notebook that works in the Data Lab environment to fully reduce example Gemini North GMOS I-band imaging data.
This is a version of the DRAGONS Jupyter notebook tutorial presented here: 
https://gitlab.com/nsf-noirlab/csdc/usngo/DRAGONS_tutorials/-/blob/main/GMOS_IM_FIELD.ipynb

This notebook will not present all of the details of the many options available to adjust or optimize the DRAGONS GMOS data reduction process, rather will just show one example of a standard reduction of a GMOS imaging dataset. 
More extensive explanations can be found in the general DRAGONS GMOS data reduction tutorial from Gemini linked here:
https://dragons.readthedocs.io/projects/gmosimg-drtutorial/en/v3.1.0/

The data used in this notebook example is GMOS I band imaging from the Gemini archive of a starry field from the Gemini North Hamamatsu CCD commissioning (Program: GN-2017A-SV-151).


<a class="anchor" id="disclaimer"></a>
# Disclaimer & attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445

* Data Lab disclaimer: https://datalab.noirlab.edu/disclaimers.php

* DRAGONS publication: Labrie et al., "DRAGONS - Data Reduction for Astronomy from Gemini Observatory North and South", ASPC, 523, 321L, https://ui.adsabs.harvard.edu/abs/2019ASPC..523..321L/abstract

* DRAGONS open source software publication: https://zenodo.org/record/7776065#.ZDg5qOzMLUI


<a class="anchor" id="imports"></a>
# Importing Python Libraries** (you'll probably have to install the `wget` and `ipympl` libraries)

In [ ]:
from __future__ import print_function

import glob
import wget

from gempy.adlibrary import dataselect
from recipe_system import cal_service
from recipe_system.reduction.coreReduce import Reduce
from gempy.utils import logutils

from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
from matplotlib.colors import LogNorm

%matplotlib inline

<a class="anchor" id="About"></a>
# About the dataset

The data used for this tutorial is a dithered sequence on a starry field.

The table below contains a summary of the dataset downloaded in the previous section:


| Observation Type | File name(s) | Purpose and Exposure (seconds) |
| :--- | :--- | :---: |
| Science | N20170614S0201-205 | 10 s, i-band |
| Bias | N20170613S0180-184 |  |
| Bias | N20170615S0534-538 |  |
| Twilight Flats | N20170702S0178-182 | 40 to 16 s, i-band |

<a class="anchor" id="Downloading_Data"></a>
# Downloading the data

In [ ]:
## uncomment the lines and run the cell

#wget.download("http://archive.gemini.edu/file/N20170613S0180.fits")
#wget.download("http://archive.gemini.edu/file/N20170613S0181.fits")
#wget.download("http://archive.gemini.edu/file/N20170613S0182.fits")
#wget.download("http://archive.gemini.edu/file/N20170613S0183.fits")
#wget.download("http://archive.gemini.edu/file/N20170613S0184.fits")
#wget.download("http://archive.gemini.edu/file/N20170614S0201.fits")
#wget.download("http://archive.gemini.edu/file/N20170614S0202.fits")
#wget.download("http://archive.gemini.edu/file/N20170614S0203.fits")
#wget.download("http://archive.gemini.edu/file/N20170614S0204.fits")
#wget.download("http://archive.gemini.edu/file/N20170614S0205.fits")
#wget.download("http://archive.gemini.edu/file/N20170615S0534.fits")
#wget.download("http://archive.gemini.edu/file/N20170615S0535.fits")
#wget.download("http://archive.gemini.edu/file/N20170615S0536.fits")
#wget.download("http://archive.gemini.edu/file/N20170615S0537.fits")
#wget.download("http://archive.gemini.edu/file/N20170615S0538.fits")
#wget.download("http://archive.gemini.edu/file/N20170702S0178.fits")
#wget.download("http://archive.gemini.edu/file/N20170702S0179.fits")
#wget.download("http://archive.gemini.edu/file/N20170702S0180.fits")
#wget.download("http://archive.gemini.edu/file/N20170702S0181.fits")
#wget.download("http://archive.gemini.edu/file/N20170702S0182.fits")

# Create and move data to raw/ directory (uncomment first)

In [ ]:
#!mkdir raw/
#!mv N*.fits raw/

<a class="anchor" id="DRAGONS_logger"></a>
# Setting up the DRAGONS logger

DRAGONS comes with a local calibration manager that uses the same calibration association rules as the Gemini Observatory Archive. This allows reduce to make requests to a local light-weight database for matching processed calibrations when needed to reduce a dataset.

This simply tells the system where to put the calibration database, the database that will keep track of the processed calibrations we are going to send to it.

In [ ]:
logutils.config(file_name='gmos_data_reduction.log')

In [ ]:
all_files = glob.glob('raw/*.fits')
all_files.sort()

<a class="anchor" id="File_Lists"></a>
# Create file lists

This data set contains science and calibration frames. For some programs, it could have different observed targets and different exposure times depending on how you like to organize your raw data.

The DRAGONS data reduction pipeline does not organize the data for you. You have to do it. DRAGONS provides tools to help you with that.

The first step is to create lists that will be used in the data reduction process. For that, we use dataselect. Please, refer to the dataselect documentation for details regarding its usage.

**List of Biases**

In [ ]:
list_of_biases = dataselect.select_data(
    all_files,
    ['BIAS'],
    []
)

**List of Flats**

If your dataset has flats obtained with more than one filter, you can add the --expr 'filter_name=="i"' expression to get only the flats obtained within the i-band. For example:

In [ ]:
list_of_flats = dataselect.select_data(
     all_files,
     ['FLAT'],
     [],
     dataselect.expr_parser('filter_name=="i"')
)

**List of Science Data**

In [ ]:
list_of_science = dataselect.select_data(
    all_files,
    [],
    ['CAL'],
    dataselect.expr_parser('(observation_class=="science" and filter_name=="i")')
)

<a class="anchor" id="Master_Bias"></a>
# Create a Master Bias
We start the data reduction by creating a master bias for the science data. It can be created and added to the calibration database using the commands below:

In [ ]:
reduce_bias = Reduce()
reduce_bias.files.extend(list_of_biases)
reduce_bias.runr()

<a class="anchor" id="Master_Flat"></a>
# Create a Master Flat Field

Twilight flats images are used to produce an imaging master flat and the result is added to the calibration database.

In [ ]:
reduce_flats = Reduce()
reduce_flats.files.extend(list_of_flats)
reduce_flats.runr()

<a class="anchor" id="Reduce_Science"></a>
# Reduce Science Images

Once we have our calibration files processed and added to the database, we can run reduce on our science data.

This command will generate bias and flat corrected files and will stack them. If a fringe frames is needed this command will apply the correction. The stacked image will have the _stack suffix.

The output stack units are in electrons (header keyword BUNIT=electrons). The output stack is stored in a multi-extension FITS (MEF) file. The science signal is in the “SCI” extension, the variance is in the “VAR” extension, and the data quality plane (mask) is in the “DQ” extension.

In [ ]:
reduce_science = Reduce()
reduce_science.files.extend(list_of_science)
reduce_science.runr()

<a class="anchor" id="Display_Image"></a>
# Display the Stacked Image

In [ ]:
image_file = "N20170614S0201_image.fits"
hdu_list = fits.open(image_file)
hdu_list.info()

In [ ]:
image_data = fits.getdata(image_file, ext=1)
print(image_data.shape)

In [ ]:
plt.figure(figsize = (10,10))
plt.imshow(image_data,cmap='gray',norm=LogNorm(vmin=0.01, vmax=1000000),origin='lower')
plt.show()

<a class="anchor" id="Clean-up"></a>
# Optional: remove duplicate calibrations and remove raw data (uncomment lines before running)

In [ ]:
#!rm -rf *_bias.fits *_flat.fits
#!rm -rf raw/